In [ ]:
from firmware_interface import ThermalMockupV2
"""
Probe 3: 0xbd3
Probe
2: 0xc0b
Probe 1: 0xbd3
"""

commands = ["Probe 3: 0xbd3", "Probe ","2: 0xc0b", "Probe 1: 0xbd3"]
probes = ["P2", "P1", "P3"]
fw_interface = ThermalMockupV2()
for command in commands:
    last_readout = "probe "
    for name in probes:
        data = command

        data = data.lower()
        validated_adc_value = None

        #first check if data was split over two lines, sometimes happens
        measure_adc_command = fw_interface.write_sensor(name)
        #print(measure_adc_command)
        merged_line_data = last_readout + data
        #print(merged_line_data)
        expected_data_length = fw_interface.data_line_length(name) #len(self.measure_adc_command) + self.raw_adc_length + 1 #+1 for the space between "measure 1" and raw_adc, total is 16 
        data_was_split = (
            (merged_line_data).count(measure_adc_command)==1 #if data was split, the merged data should only contain the measure_adc_command once
            and 
            merged_line_data.startswith(measure_adc_command) #cleans rare case where previous cut off is same length as current cut off
            and
            len(merged_line_data)==expected_data_length #its length should also be something like len("measure 1 72a4ff"), prevents cases like "measure 1 72a4ffmeasure 2 72a4ff"
        )
        if data_was_split:
            print("data was split", "merged_line_data", merged_line_data)
            data = merged_line_data

        #check if command is in data
        if measure_adc_command in data and len(data) == expected_data_length:
            raw_adc = fw_interface.read_sensor(data)
            if raw_adc != '0' or not raw_adc:
                #sometimes raw_adc can give 0, skip append for these
                validated_adc_value = raw_adc

        #for checking split lines on arduino
        last_readout = data

        if validated_adc_value is not None:
            print(f"readout for: {name}")
            print(validated_adc_value)



readout for: P3
0xbd3
readout for: P2
0xc0b
readout for: P1
0xbd3
